In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix

In [8]:
df = pd.read_csv('train.csv')

In [11]:
X = df.drop('labels',axis = 1).to_numpy()
Y = df['labels']
X.shape

(5250, 1200)

In [13]:
result_X = X.reshape(-1,20,20,3)
result_X.shape

(5250, 20, 20, 3)

In [14]:
from sklearn.model_selection import train_test_split
train,test,Y_train,Y_tests = train_test_split(result_X,Y, test_size=0.2, random_state = 20113046)
train.shape,test.shape,Y_train.shape

((4200, 20, 20, 3), (1050, 20, 20, 3), (4200,))

In [15]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [16]:
nRows,nCols,nDims = train.shape[1:]
train_data = train.reshape(train.shape[0], nRows, nCols, nDims)
test_data = test.reshape(test.shape[0], nRows, nCols, nDims)
input_shape = (nRows, nCols, nDims)

train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

In [17]:
train_data /= 255
test_data /= 255

train_labels_one_hot = to_categorical(Y_train)
test_labels_one_hot = to_categorical(Y_tests)

In [19]:
def createModel():
    model = Sequential()
    # The first two layers with 32 filters of window size 3x3
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(1, 1)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))
    
    return model

In [20]:
model1 = createModel()
batch_size = 32
epochs = 50
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 20, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 18, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 18, 18, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 18, 18, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                        

In [22]:
model1.fit(train, Y_train, epochs=25, batch_size=32)

Epoch 1/25
132/132 [==============================] - 52s 363ms/step - loss: 0.4939 - accuracy: 0.2643
Epoch 2/25
132/132 [==============================] - 46s 347ms/step - loss: 0.3951 - accuracy: 0.2643
Epoch 3/25
132/132 [==============================] - 46s 352ms/step - loss: 0.3436 - accuracy: 0.2643
Epoch 4/25
132/132 [==============================] - 46s 345ms/step - loss: 0.3085 - accuracy: 0.2643
Epoch 5/25
132/132 [==============================] - 45s 344ms/step - loss: 0.2792 - accuracy: 0.2643
Epoch 6/25
132/132 [==============================] - 47s 354ms/step - loss: 0.2562 - accuracy: 0.2643
Epoch 7/25
132/132 [==============================] - 45s 344ms/step - loss: 0.2254 - accuracy: 0.2643
Epoch 8/25
132/132 [==============================] - 46s 348ms/step - loss: 0.2001 - accuracy: 0.2643
Epoch 9/25
132/132 [==============================] - 45s 344ms/step - loss: 0.1753 - accuracy: 0.2643
Epoch 10/25
132/132 [==============================] - 41s 313ms/step - l

In [24]:

# predict probabilities for test set
yhat_probs = model1.predict(test, verbose=0)
# predict crisp classes for test set
# yhat_classes = model1.predict_classes(test, verbose=0)

In [25]:
yhat_probs = yhat_probs[:, 0]
#yhat_classes = yhat_classes[:, 0]

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [27]:
accuracy = accuracy_score(Y_tests, yhat_probs)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(Y_tests, yhat_probs)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(Y_tests, yhat_probs)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(Y_tests, yhat_probs)
print('F1 score: %f' % f1)

Accuracy: 0.276190
Precision: 0.276190
Recall: 1.000000
F1 score: 0.432836
